In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import time

### Chart Data Remove Duplicate & Output Ids

In [13]:
data = pd.read_csv("charts.csv")
data

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0
...,...,...,...,...,...,...,...,...,...
26173509,BYE,46,2021-07-31,Jaden,https://open.spotify.com/track/3OUyyDN7EZrL7i0...,Vietnam,viral50,MOVE_UP,NaN
26173510,Pillars,47,2021-07-31,My Anh,https://open.spotify.com/track/6eky30oFiQbHUAT...,Vietnam,viral50,NEW_ENTRY,NaN
26173511,Gái Độc Thân,48,2021-07-31,Tlinh,https://open.spotify.com/track/2klsSb2iTfgDh95...,Vietnam,viral50,MOVE_DOWN,NaN
26173512,Renegade (feat. Taylor Swift),49,2021-07-31,Big Red Machine,https://open.spotify.com/track/1aU1wpYBSpP0M6I...,Vietnam,viral50,MOVE_DOWN,NaN


In [35]:
songs = data.drop_duplicates(subset=['title'])
songs.reset_index(inplace=True, drop=True)
songs

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0
...,...,...,...,...,...,...,...,...,...
164802,めちゃくちゃ,50,2021-07-27,vvpskvd.,https://open.spotify.com/track/0u0ipbfDa5uKTdG...,Ukraine,viral50,NEW_ENTRY,NaN
164803,Gris Atardecer,20,2021-07-28,Los Visitantes,https://open.spotify.com/track/5vsdlfsRw5SEayU...,Argentina,viral50,NEW_ENTRY,NaN
164804,Imparfait inconnu,28,2021-07-28,Adé,https://open.spotify.com/track/3W7x967esdawMwz...,France,viral50,NEW_ENTRY,NaN
164805,Still Holy (feat. Ryan Ofei & Naomi Raine),47,2021-07-28,"Tribl, Maverick City Music",https://open.spotify.com/track/4Cw36mfQ1z4JnjF...,South Africa,viral50,NEW_ENTRY,NaN


In [36]:
list(songs.columns)

['title',
 'rank',
 'date',
 'artist',
 'url',
 'region',
 'chart',
 'trend',
 'streams']

In [38]:
songs['date_formatted'] = pd.to_datetime(songs['date'], format="%Y-%m-%d")
songs['year'] = songs['date_formatted'].dt.year
songs

C:\Users\12560\AppData\Local\Temp\ipykernel_5496\2122764647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songs['date_formatted'] = pd.to_datetime(songs['date'], format="%Y-%m-%d")
C:\Users\12560\AppData\Local\Temp\ipykernel_5496\2122764647.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songs['year'] = songs['date_formatted'].dt.year


,title,rank,date,artist,url,region,chart,trend,streams,date_formatted,year
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,2017-01-01,2017
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,2017-01-01,2017
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,2017-01-01,2017
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,2017-01-01,2017
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,2017-01-01,2017
...,...,...,...,...,...,...,...,...,...,...,...
164802,めちゃくちゃ,50,2021-07-27,vvpskvd.,https://open.spotify.com/track/0u0ipbfDa5uKTdG...,Ukraine,viral50,NEW_ENTRY,NaN,2021-07-27,2021
164803,Gris Atardecer,20,2021-07-28,Los Visitantes,https://open.spotify.com/track/5vsdlfsRw5SEayU...,Argentina,viral50,NEW_ENTRY,NaN,2021-07-28,2021
164804,Imparfait inconnu,28,2021-07-28,Adé,https://open.spotify.com/track/3W7x967esdawMwz...,France,viral50,NEW_ENTRY,NaN,2021-07-28,2021
164805,Still Holy (feat. Ryan Ofei & Naomi Raine),47,2021-07-28,"Tribl, Maverick City Music",https://open.spotify.com/track/4Cw36mfQ1z4JnjF...,South Africa,viral50,NEW_ENTRY,NaN,2021-07-28,2021


In [60]:
for year in pd.unique(songs['year']):
    url = songs[songs['year'] == year][['title','date_formatted','url']]
    url.reset_index(inplace=True, drop=True)
    url.to_csv('Songs URL {}.csv'.format(year), index=False)

### Credential

In [4]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

# playlist_index = 0

# playlists = json.load(open('playlist_like_dislike.json'))
# playlist_uri = playlists[playlist_index]['uri']
# like = playlists[playlist_index]['like']

In [5]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp

### Read URLs

In [6]:
years = [2017, 2018, 2019, 2020, 2021]
url_years = {}
for year in years:
    df = pd.read_csv("Data/Songs URL {}.csv".format(year))
    ##Need to dynamically change path here
    df = pd.read_csv("./Data/Datasets_202303/Songs URL {}.csv".format(year))
    df = pd.DataFrame(df)
    df['id'] = df.apply(lambda x: x['url'].split("/")[-1],axis=1)
    url_years[year] = df

### 1.1 Tracks Information
To avoid failure due to connection problems, etc. pull the data one year after one year

#### 2017

In [8]:
year = 2017
url = url_years[year]
len(url)

40111

In [49]:
tracks = []
for i in range(0, len(url), 50):
    try:
        if (i+50) < len(url): 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
    except:
            client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
            sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
    finally:
        if (i+50) < len(url): 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
        tracks += info

In [50]:
tracks_df = pd.DataFrame(tracks)
tracks_df.to_csv('tracks info {}.csv'.format(year))
print('{} done'.format(year))

2017 done


#### 2018

In [5]:
year = 2018
url = url_years[year]
len(url)

36596

In [6]:
tracks = []
for i in range(0, 20000, 50):
    try:
        if (i+50) < 20000: 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
    except:
            client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
            sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
    finally:
        if (i+50) < len(url): 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
        tracks += info

In [8]:
for i in range(20000, len(url), 50):
    try:
        if (i+50) < len(url): 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
    except:
            client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
            sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
    finally:
        if (i+50) < len(url): 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
        tracks += info

In [9]:
tracks_df = pd.DataFrame(tracks)
tracks_df.to_csv('tracks info {}.csv'.format(year))
print('{} done'.format(year))

2018 done


#### 2019

In [5]:
year = 2019
url = url_years[year]
len(url)

27658

In [8]:
tracks = []
for i in range(0, len(url), 50):
    try:
        if (i+50) < len(url): 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
    except:
            client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
            sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
    finally:
        if (i+50) < len(url): 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
        tracks += info

In [9]:
tracks_df = pd.DataFrame(tracks)
tracks_df.to_csv('tracks info {}.csv'.format(year))
print('{} done'.format(year))

2019 done


#### 2020

In [52]:
url_2020 = url_years[2020]
tracks_info = []
for i in range(0, 10000, 50):
    if (i+50) < len(url_2020): 
        tracks_info += sp.tracks(list(url_2020.iloc[i:(i+50)]['id']))['tracks']
    else:
        tracks_info += sp.tracks(list(url_2020.iloc[i:]['id']))['tracks']

In [53]:
pd.DataFrame(tracks_info).to_csv("sample_info.csv")

In [56]:
tracks = []
for i in range(0, len(url_2020), 50):
    try:
        if (i+50) < len(url_2020): 
            info = sp.tracks(list(url_2020.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url_2020.iloc[i:]['id']))['tracks']
    except:
            client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
            sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
    finally:
        if (i+50) < len(url_2020): 
            info = sp.tracks(list(url_2020.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url_2020.iloc[i:]['id']))['tracks']
        tracks += info

In [62]:
tracks_df = pd.DataFrame(tracks)
tracks_df.to_csv('tracks info 2020.csv')

#### 2021

In [9]:
year = 2021
url = url_years[year]
len(url)

28024

In [52]:
tracks = []
for i in range(0, len(url), 50):
    try:
        if (i+50) < len(url): 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
    except:
            client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
            sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
    finally:
        if (i+50) < len(url): 
            info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
        else:
            info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
        tracks += info

In [53]:
tracks_df = pd.DataFrame(tracks)
tracks_df.to_csv('tracks info {}.csv'.format(year))
print('{} done'.format(year))

2021 done


### 1.2 Track Clear Information
Only Retain the information we need.
[id, name, explicit, number of available markets, 

In [3]:
year = 2017
tracks = pd.read_csv(f"data/tracks info {year}.csv")
tracks

,Unnamed: 0,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,0,"{'album_group': 'album', 'album_type': 'album'...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,195840,False,{'isrc': 'USSD11600299'},{'spotify': 'https://open.spotify.com/track/6m...,https://api.spotify.com/v1/tracks/6mICuAdrwEjh...,6mICuAdrwEjh6Y6lroV2Kg,False,Chantaje (feat. Maluma),75,https://p.scdn.co/mp3-preview/83f093a4caa3d8b8...,3,track,spotify:track:6mICuAdrwEjh6Y6lroV2Kg
1,1,"{'album_group': 'single', 'album_type': 'singl...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,259195,False,{'isrc': 'USSD11600252'},{'spotify': 'https://open.spotify.com/track/7D...,https://api.spotify.com/v1/tracks/7DM4BPaS7uof...,7DM4BPaS7uofFul3ywMe46,False,Vente Pa' Ca (feat. Maluma),69,https://p.scdn.co/mp3-preview/616bedd27ef135cb...,1,track,spotify:track:7DM4BPaS7uofFul3ywMe46
2,2,"{'album_group': 'album', 'album_type': 'album'...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BG', 'CA', 'CL...",1,222560,False,{'isrc': 'USSD11600135'},{'spotify': 'https://open.spotify.com/track/3A...,https://api.spotify.com/v1/tracks/3AEZUABDXNte...,3AEZUABDXNtecAOSC1qTfo,False,Reggaetón Lento (Bailemos),70,https://p.scdn.co/mp3-preview/83f163d3b6506f8f...,3,track,spotify:track:3AEZUABDXNtecAOSC1qTfo
3,3,"{'album_group': 'album', 'album_type': 'album'...",[{'external_urls': {'spotify': 'https://open.s...,[],1,205600,False,{'isrc': 'USUM71604778'},{'spotify': 'https://open.spotify.com/track/6r...,https://api.spotify.com/v1/tracks/6rQSrBHf7HlZ...,6rQSrBHf7HlZjtcMZ4S4bO,False,Safari,0,NaN,3,track,spotify:track:6rQSrBHf7HlZjtcMZ4S4bO
4,4,"{'album_group': 'single', 'album_type': 'singl...",[{'external_urls': {'spotify': 'https://open.s...,[],1,234320,False,{'isrc': 'US2BU1600020'},{'spotify': 'https://open.spotify.com/track/58...,https://api.spotify.com/v1/tracks/58IL315gMSTD...,58IL315gMSTD37DOZPJ2hf,False,Shaky Shaky,0,NaN,1,track,spotify:track:58IL315gMSTD37DOZPJ2hf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40106,40106,"{'album_group': 'single', 'album_type': 'singl...",[{'external_urls': {'spotify': 'https://open.s...,[],1,199412,False,{'isrc': 'US5TA1700045'},{'spotify': 'https://open.spotify.com/track/1q...,https://api.spotify.com/v1/tracks/1q0Qj0PW7qsf...,1q0Qj0PW7qsfaEvZE0wWsV,False,Hold Me Tight,0,NaN,4,track,spotify:track:1q0Qj0PW7qsfaEvZE0wWsV
40107,40107,"{'album_group': 'single', 'album_type': 'singl...",[{'external_urls': {'spotify': 'https://open.s...,[],1,230634,False,{'isrc': 'GBER71700223'},{'spotify': 'https://open.spotify.com/track/3x...,https://api.spotify.com/v1/tracks/3xesmCtVNLeZ...,3xesmCtVNLeZiXjNcfs6Tf,False,Sex Music,0,NaN,1,track,spotify:track:3xesmCtVNLeZiXjNcfs6Tf
40108,40108,"{'album_group': 'single', 'album_type': 'singl...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,189385,False,{'isrc': 'NZRI11701610'},{'spotify': 'https://open.spotify.com/track/05...,https://api.spotify.com/v1/tracks/05EypeGQ9wez...,05EypeGQ9wezXtrsgUKjKc,False,What Do,3,https://p.scdn.co/mp3-preview/a6e0d80fcdbec1f7...,1,track,spotify:track:05EypeGQ9wezXtrsgUKjKc
40109,40109,"{'album_group': 'compilation', 'album_type': '...",[{'external_urls': {'spotify': 'https://open.s...,[],1,252213,False,{'isrc': 'PT6GJ1700003'},{'spotify': 'https://open.spotify.com/track/4p...,https://api.spotify.com/v1/tracks/4pxPlfILTAdq...,4pxPlfILTAdqETp6FCfqj7,False,Fim do mundo (E ao cabo do teu ser),0,NaN,2,track,spotify:track:4pxPlfILTAdqETp6FCfqj7


In [14]:
tracks.loc[0,'available_markets'].replace('[','').replace(']','').replace("'",'').replace(' ', '').split(',')
# tracks = tracks[['id', 'name', 'popularity', '']]

184

In [17]:
def compute_markets(s):
    markets = s.replace('[','').replace(']','').replace("'",'').replace(' ', '').split(',')
    if markets[0] == '':
        return 0
    else:
        return len(markets)

In [48]:
years = [2017, 2018, 2019, 2020, 2021]
for year in years:
    tracks = pd.read_csv(f"data/tracks info {year}.csv")
    cat = pd.unique(tracks['explicit'])
    tracks['num_markets'] = tracks['available_markets'].apply(compute_markets)
    tracks['explicit_trans'] = None 
    tracks['explicit_trans'][tracks['explicit'] == cat[0]] = 0
    tracks['explicit_trans'][tracks['explicit'] == cat[1]] = 1
    tracks = tracks[['id','name','popularity','explicit_trans','num_markets']]
    tracks.rename(columns={'num_markets':'available_markets', 'explicit_trans':'explicit'}, inplace=True)
    tracks.to_csv(f'Data/tracks clear info {year}.csv')

In [49]:
tracks

,id,name,popularity,explicit,available_markets
0,3Ys2PYl1wyPKQIwyqhP9cQ,Stay Gold,62,0,184
1,3CLSHJv5aUROAN2vfOyCOh,Close Eyes,80,1,184
2,4bF2y8rR7GUjrd5LNEvej3,París,67,0,184
3,7urtwZEtOypb3r8HNi42dT,Duvet,0,0,0
4,2K1CQMoG2Dy5nmeOYnhoi8,Poblado - Remix,60,0,184
...,...,...,...,...,...
28019,0u0ipbfDa5uKTdGWvVNNzP,めちゃくちゃ,46,1,184
28020,5vsdlfsRw5SEayUQaUR5RJ,Gris Atardecer,0,0,0
28021,3W7x967esdawMwzvMsPmEs,Imparfait inconnu,20,0,184
28022,4Cw36mfQ1z4JnjFOEtX4hP,Still Holy (feat. Ryan Ofei & Naomi Raine),37,0,184


### 2. Audio Features

In [47]:
for year in years:
    url = url_years[year]
    audio_feat=[]
    for i in range(0, len(url), 100):
        if (i+100) < len(url): 
            audio_feat += list(sp.audio_features(list(url.iloc[i:(i+100)]['id'])))
        else:
            audio_feat += list(sp.audio_features(list(url.iloc[i:]['id'])))
    # Deal with None objects
    audio_feat = list(filter(lambda x: x is not None, audio_feat))
    audio_feat_df = pd.DataFrame(audio_feat)
    audio_feat_df.to_csv('audio feat {}.csv'.format(year))
    print("{} done".format(year))

2017 done
2018 done
2019 done
2020 done
2021 done


### Try artist & album

#### Retrieve data

In [ ]:
years = [2017, 2018, 2019, 2020, 2021]
# album: [id, name, total_available_markets, total_tracks, genres, popularity]
# artist: [id, name, number of total followers, genres, popularity]

for year in years:
    url = url_years[year]
    print(len(url))
    tid = []
    albums = []
    artists = []
    for i in range(0, len(url), 50):
        try:
            if (i+50) < len(url): 
                info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
            else:
                info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
        except:
                client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
                sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
        finally:
            if (i+50) < len(url): 
                info = sp.tracks(list(url.iloc[i:(i+50)]['id']))['tracks']
            else:
                info = sp.tracks(list(url.iloc[i:]['id']))['tracks']
            for t in info:
                tid.append(t['id'])
                albums.append(t['album']['id'])
                artists.append([x['id'] for x in t['artists']])
    pd.DataFrame({'track_id':tid, 'album_id': albums, 'artists_id': artists}).to_csv(f'Data/ids {year}.csv')
    print(f"{year} done")  

In [ ]:
for yr, url in url_years.items():
    if yr == 2017 or yr == 2018 or yr == 2020:
        continue
    metas = pd.DataFrame(columns=['id','analyzer_version', 'platform', 'detailed_status', 'status_code', 'timestamp', 'analysis_time', 'input_process'])
    tracks = pd.DataFrame(columns=['id','num_samples', 'duration', 'sample_md5', 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'])
    bars = pd.DataFrame(columns=['id','start', 'duration', 'confidence'])
    beats = pd.DataFrame(columns=['id','start', 'duration', 'confidence'])
    sections = pd.DataFrame(columns=['id','start', 'duration', 'confidence', 'loudness', 'tempo', 'tempo_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'time_signature', 'time_signature_confidence'])
    segments = pd.DataFrame(columns=['id','start', 'duration', 'confidence', 'loudness_start', 'loudness_max_time', 'loudness_max', 'loudness_end', 'pitches', 'timbre'])
    tatums = pd.DataFrame(columns=['id','start', 'duration', 'confidence'])
    print(f'{yr} audio analysis')
    for i in tqdm(range(len(url))):
        if i%1000 ==0:
            time.sleep(5)
            client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
            sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
# for yr, url in url_years.items():
#     metas = pd.DataFrame(columns=['id','analyzer_version', 'platform', 'detailed_status', 'status_code', 'timestamp', 'analysis_time', 'input_process'])
#     tracks = pd.DataFrame(columns=['id','num_samples', 'duration', 'sample_md5', 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'])
#     bars = pd.DataFrame(columns=['id','start', 'duration', 'confidence'])
#     beats = pd.DataFrame(columns=['id','start', 'duration', 'confidence'])
#     sections = pd.DataFrame(columns=['id','start', 'duration', 'confidence', 'loudness', 'tempo', 'tempo_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'time_signature', 'time_signature_confidence'])
#     segments = pd.DataFrame(columns=['id','start', 'duration', 'confidence', 'loudness_start', 'loudness_max_time', 'loudness_max', 'loudness_end', 'pitches', 'timbre'])
#     tatums = pd.DataFrame(columns=['id','start', 'duration', 'confidence'])
#     print(f'{yr} audio analysis')
#     for i in tqdm(range(len(url))):
#         if i%1000 ==0:
#             time.sleep(5)
#             client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
#             sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
    
#         song_id = url.iloc[i]['id']
#         try:
#             analysis = sp.audio_analysis(song_id)
        
#             meta = pd.DataFrame([analysis['meta']])
#             meta['id'] = song_id
#             metas = pd.concat([metas, meta])
            
#             track = pd.DataFrame([analysis['track']])
#             track['id'] = song_id
#             tracks = pd.concat([tracks, track])
            
#             bar = pd.DataFrame(analysis['bars'])
#             bar['id'] = song_id
#             bars = pd.concat([bars, bar])
            
#             beat = pd.DataFrame(analysis['beats'])
#             beat['id'] = song_id
#             beats = pd.concat([beats, beat])

#             section = pd.DataFrame(analysis['sections'])
#             section['id'] = song_id
#             sections = pd.concat([sections, section])
                
#             segment = pd.DataFrame(analysis['segments'])
#             segment['id'] = song_id
#             segments = pd.concat([segments, segment])

#             tatum = pd.DataFrame(analysis['tatums'])
#             tatum['id'] = song_id
#             tatums = pd.concat([tatums, tatum])        
#         except:
#             time.sleep(5)
#             client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
#             sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5, retries=5)
#             continue
    
#     metas.to_csv(f'./Data/Datasets_202303/Metas {yr}.csv', index=False)
    
#     tracks.to_csv(f'./Data/Datasets_202303/Tracks {yr}.csv', index=False)
    
#     bars.to_csv(f'./Data/Datasets_202303/Bars {yr}.csv', index=False)
    
#     beats.to_csv(f'./Data/Datasets_202303/Beats {yr}.csv', index=False)
    
#     sections.to_csv(f'./Data/Datasets_202303/Sections {yr}.csv', index=False)
    
#     segments.to_csv(f'./Data/Datasets_202303/Segments {yr}.csv', index=False)
    
#     tatums.to_csv(f'./Data/Datasets_202303/Tatums {yr}.csv', index=False)

Found that the data in audio analysis not that useful for our model.